# LLM Engineer's Handbook
- Iusztin, Paul / Labonne, Maxime. **LLM Engineer's Handbook: Master the art of engineering large language models from concept to production**. 2024. Packt. - [code](https://github.com/PacktPublishing/LLM-Engineers-Handbook)

# Running example

LLM Twin: an AI character that incorporates your writing style, voice, and personality into an LLM.

# Terms

- FTI: feature/training/inference

# Architecture

Figure 1.6: LLM Twin high-level architecture
<img src="https://static.packt-cdn.com/products/9781836200079/graphics/Images/B31105_01_06.png"/>

# Tools

- Python 3.11.8
- pyenv
- Poetry: dependency and virtual environment management
- Peo the Poet: a plugin on top of Poetry to manage and execute all the CLI commands required to interact with the project.
- Docker: 27.1.1+
- Huggine Face: model registry
- ZenML: orchestrator, artifacts, and metadata
- Comet ML: experiment tracker
  - Opik: prompt monitoring 
- MongoDB: NoSQL database
- Qdrant: vector database
- AWS
  - S3: object storage
  - ECR: container registry
  - SagaMaker: compute for training and inference 

# Data collection pipeline

ETL
- extract: Medium, Substack, GitHub - crawler
  - Scrapy, Crawl4AI
- transform
- load
  - ORM: SQLAlchemy, SQLModel(FastAPI)
  - ODM
    - Pydantic: generic module

Figure 3.1: LLM Twin’s data collection ETL pipeline architecture


data: `data/data_warehouse_raw_data`

# Feature pipeline

## RAG

- Retrieval: search for relevant data
- Augmented: add the data as context to the prompt
- Generation: use the augmented prompt with an LLM for generation

system models:
- ingestion pipeline: a batch or streaming pipeplie used to populate the vector DB
- retrieval pipeline: a module that queries the vector DB and retrieves relevant entries to the user's input
- generation pipeline: the layer that uses the retrieved data to augment the prompt and an LLM to generate answers

Figure 4.1: Vanilla RAG architecture

<img src="./images/Vanilla RAG architecture.png" width="600"/>

RAG ingestion pipeline:
- data extraction module: 从多个源(例如数据库, API或网页)收集必须奥的数据.
- cleaning layer: 标准化提取的数据, 移除不需要的特征.
- chunking module: 拆分清洗后的文档, 不超过嵌入模型的最大输入大小, 按语义相关的拆分.
- embedding component: 使用chunk的内容(文本, 图片, 语音等)输入一个嵌入模型, 映射到带有语义值的稠密向量上
- loading module: 使用嵌入的chunk和元数据文档, 嵌入作为查询相似chunk的索引, 元数据用于访问用以增强提示的信息.

RAG retrieval pipeline:
- 接收用户输入: 文本, 图片, 语音等
- 内嵌
- 在向量数据库中查询与用户输入向量相似的向量.

RAG generation pipeline:
- 使用用户输入, 检索出的数据, 传入LLM, 生成答案.
- 最终的提示依赖于系统, 用户查询生成的提示模板(prompt template)和检索出的上下文.

Embedding
- early text data: Word2Vec, Glove
- encoder-only transformers: BERT, RoBERTa
- Python packages:
  - Sentence Transformers
  - Hugging Face's transformer
- images: CNN(Convolutional Neural Networks) 

vector DB
- algorithms: ANN(approximate nearest neighbor) to find the close neighbors
- workflow
  - indexing vectors
  - querying for similarity
  - post-processing results

batch v.s. streaming

Figure 4.10: Tools on the streaming versus batch and smaller versus bigger data spectrum

<img src="./images/Batch vs Streaming tools.png" width="400"/>

# Training pipeline

## SFT: Supervised Fine-Tuning

- [Differences between Pre-Training and Supervised Fine-Tuning (SFT)](https://techcommunity.microsoft.com/blog/machinelearningblog/differences-between-pre-training-and-supervised-fine-tuning-sft/4220673)
- [Understanding and Using Supervised Fine-Tuning (SFT) for Language Models](https://cameronrwolfe.substack.com/p/understanding-and-using-supervised) - 2023-08-11
  - SFT: Supervised Fine-Tuning
  - ELHF: rEinforcement Learning from Human Feedback

<img src="./images/Different stages of training for an LLM.png" width="600"/>

SFT: Supervised Fine-Tuning
- 在初始的预训练阶段(LLM学习预测序列中下一个token)之后, SFT使用精心准备的指令(instruction)和相应的答案对, 来refine(提炼/完善)模型的能力.
- 填充缝隙: 模型的通用语言理解, 模型的实际效用(utility)

创建指令数据集

post-training data pipeline
- data curation: (对数字信息的)综合处理，指对数字信息的选择，保存，管理等一系列处理行为，其目的是建立完善的查询机制和提供可靠的参考数据
  - task-specific model: 从既有数据库中收集预期任务的示例, 或创建新的
  - domain-specific model: 需要领域专家参与
- data deduplication
- data decontamination: 净化/去污
- deta quality evaluation
- data exploration
- data generation
- data augmentation

SFT技术
- 需要更多的控制: know your data. 可定制性: the fine-tuned model is unique
- 指令数据集格式: Alpaca, ShareGPT, OpenAI
- 聊天模板chat templates: ChatML, Llama 3, Mistral, ...
- 技术: full fine-tuning, LoRA, QLoRA
- tranning parameters: hyperparameters
  - learning rate
  - batch size
  - maximum sequence length, packing
  - number of epochs
  - optimizers
  - weight decay
  - gradient checkpointing

## Preference alignment
- 引入直接的人类或AI反馈到训练过程中.

DPO: Direct Preference Optimization - [paper](https://arxiv.org/abs/2305.18290)

preference data: a collection of responses to a given instruction, ranked by humans or language models.

## Evaluating LLMs

LLM evaluation forms:
- multiple-choice question answering
- open-ended instructions
- feedback from real users
- general-purpose evaluation

model evaluation
- without any prompt engineering, RAG pipeline

models: 
- general-purpose
- domain-specific
- task-specific

RAG evaluation
- retrieval accuracy
- integration quality
- factuality and relevance

Ragas: RAG Assessment

ARES: an automated evaluation framework for RAG systems

# Inference pipeline

## Inference Optimization

inference process metrics:
- latency: the time it takes to generate the first token
- throughput: the number of tokens generated per second
- memory footprint

optimization methods:
- speculative decoding
- model parallelism
- weight quantization

inference engines:
- Text Generation Inference
- vLLM
- TensorRT-LLM

## RAG

<img src="./images/RAG inference pipeline architecture.png" width="800"/>

- user query
- query expansion: 查询扩展, 反应原始用户查询的不同解释的方面
- self-quering: 从原始查询中提取有用的元数据, 例如作者名称, 作为向量搜索操作的过滤器

## Deployment

types:
- online real-time inference
- asynchronous inference
- offline batch transform

model serving:
- monolith
- microservices

<img src="./images/Microservice deployment architecture of the LLM inference pipeline.png" width="800"/>

# LLMOps

LLMOps is built on top of MLOps, which is built on DevOps.

LLMOps focuses:
- prompt monitoring and versioning
- input and output guardrails to prevent toxic behavior
- feedback loops to gather fine-tuning data
- scaling issues:
  - collect trillions of tokens for training datasets
  - train models on massive GPU clusters
  - reduce infrastructure costs

example:
- CI/CD pipeline: GitHub Actions
- CT, alerting: ZenML
- monitoring: Opik from Comet ML

infrastructure flow:

<img src="./images/Infrastructure flow.png" width="800"/>

CT pipeline:

<img src="./images/CT pipeline.png" width="800"/>